In [ ]:
from resnet1d import ResNet1D, MyDataset

In [ ]:
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

from resnet1d import ResNet1D, MyDataset

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

import pandas as pd


In [ ]:
train = pd.read_csv('mitbih_train.csv', header=None)
test = pd.read_csv('mitbih_test.csv', header=None)

In [ ]:
print(train.shape)

(87554, 188)


In [ ]:
train[187]=train[187].astype(int)
equilibre=train[187].value_counts()
print(equilibre)

0    72471
4     6431
2     5788
1     2223
3      641
Name: 187, dtype: int64


In [ ]:
from sklearn.utils import resample
df_1=train[train[187]==1]
df_2=train[train[187]==2]
df_3=train[train[187]==3]
df_4=train[train[187]==4]
df_0=(train[train[187]==0]).sample(n=20000,random_state=42)

df_1_upsample=resample(df_1,replace=True,n_samples=20000,random_state=123)
df_2_upsample=resample(df_2,replace=True,n_samples=20000,random_state=124)
df_3_upsample=resample(df_3,replace=True,n_samples=20000,random_state=125)
df_4_upsample=resample(df_4,replace=True,n_samples=20000,random_state=126)

train=pd.concat([df_0,df_1_upsample,df_2_upsample,df_3_upsample,df_4_upsample])

In [ ]:
equilibre=train[187].value_counts()
print(equilibre)

4    20000
3    20000
2    20000
1    20000
0    20000
Name: 187, dtype: int64


In [ ]:
print(train.shape)
x_train_org = train.iloc[:,:-1].to_numpy()
x_train = np.reshape(x_train_org, (len(x_train_org), 1, len(x_train_org[0])))
print(x_train.shape)
y_train = train.iloc[:,-1].to_numpy()
num_classes = len(set(y_train))
print(y_train.shape)

(87553, 188)
(87553, 1, 187)
(87553,)


In [ ]:
x_test_org = test.iloc[:,:-1].to_numpy()
x_test = np.reshape(x_test_org, (len(x_test_org), 1, len(x_test_org[0])))
y_test = test.iloc[:,-1].to_numpy()
print(x_test.shape)
print(y_test.shape)

(21891, 1, 187)
(21891,)


In [ ]:
dataset = MyDataset(x_train, y_train)
dataset_test = MyDataset(x_test, y_test)

In [ ]:
batch_size = 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, drop_last=True)

In [ ]:
device_str = "cuda"
device = torch.device(device_str if torch.cuda.is_available() else "cpu")
kernel_size = 16
stride = 2
n_block = 48
downsample_gap = 6
increasefilter_gap = 12
model = ResNet1D(
    in_channels=1, 
    base_filters=64,
    kernel_size=kernel_size, 
    stride=stride, 
    groups=32, 
    n_block=n_block, 
    n_classes=num_classes, 
    downsample_gap=downsample_gap, 
    increasefilter_gap=increasefilter_gap, 
    use_do=True)
model.to(device)

summary(model, (x_train.shape[1], x_train.shape[2]), device=device_str)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 64, 187]           1,088
   MyConv1dPadSame-2              [-1, 64, 187]               0
       BatchNorm1d-3              [-1, 64, 187]             128
              ReLU-4              [-1, 64, 187]               0
            Conv1d-5              [-1, 64, 187]           2,112
   MyConv1dPadSame-6              [-1, 64, 187]               0
       BatchNorm1d-7              [-1, 64, 187]             128
              ReLU-8              [-1, 64, 187]               0
           Dropout-9              [-1, 64, 187]               0
           Conv1d-10              [-1, 64, 187]           2,112
  MyConv1dPadSame-11              [-1, 64, 187]               0
       BasicBlock-12              [-1, 64, 187]               0
      BatchNorm1d-13              [-1, 64, 187]             128
             ReLU-14              [-1, 

In [ ]:
model.verbose = False
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
loss_func = torch.nn.CrossEntropyLoss()

In [ ]:
def accuracy(preds, y):
    _, predicted = torch.max(preds,1)
    correct = (predicted == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
n_epoch = 20
step = 0
epoch = 1
for _ in tqdm(range(n_epoch), desc="epoch", leave=False):

    # train
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    prog_iter = tqdm(dataloader, desc="Training", leave=False)
    for batch_idx, batch in enumerate(prog_iter):

        input_x, input_y = tuple(t.to(device) for t in batch)
        pred = model(input_x)
        loss = loss_func(pred, input_y)
        acc = accuracy(pred, input_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        step += 1

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    train_loss = epoch_loss / len(prog_iter)
    train_acc = epoch_acc / len(prog_iter)

    scheduler.step(_)
                
    # test
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    prog_iter_test = tqdm(dataloader_test, desc="Testing", leave=False)
    all_pred_prob = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(prog_iter_test):
            input_x, input_y = tuple(t.to(device) for t in batch)
            pred = model(input_x)
            loss = loss_func(pred, input_y)
            acc = accuracy(pred, input_y)
            all_pred_prob.append(pred.cpu().data.numpy())
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    valid_loss = epoch_loss / len(prog_iter_test)
    valid_acc = epoch_acc / len(prog_iter_test)
    print(f'Epoch: {epoch:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    epoch += 1

Epoch: 01
	Train Loss: 0.315 | Train Acc: 91.12%
	 Val. Loss: 0.325 |  Val. Acc: 91.42%


Epoch: 02
	Train Loss: 0.154 | Train Acc: 95.79%
	 Val. Loss: 0.150 |  Val. Acc: 96.56%


Epoch: 03
	Train Loss: 0.127 | Train Acc: 96.62%
	 Val. Loss: 0.122 |  Val. Acc: 96.96%


Epoch: 04
	Train Loss: 0.114 | Train Acc: 96.86%
	 Val. Loss: 0.102 |  Val. Acc: 97.35%


Epoch: 05
	Train Loss: 0.105 | Train Acc: 97.09%
	 Val. Loss: 0.110 |  Val. Acc: 97.22%


...

Epoch: 16
	Train Loss: 0.072 | Train Acc: 97.94%
	 Val. Loss: 0.074 |  Val. Acc: 98.02%


Epoch: 17
	Train Loss: 0.071 | Train Acc: 98.01%
	 Val. Loss: 0.073 |  Val. Acc: 98.02%


Epoch: 18
	Train Loss: 0.069 | Train Acc: 98.06%
	 Val. Loss: 0.073 |  Val. Acc: 98.04%


Epoch: 19
	Train Loss: 0.069 | Train Acc: 98.01%
	 Val. Loss: 0.074 |  Val. Acc: 98.08%


Epoch: 20
	Train Loss: 0.069 | Train Acc: 98.06%
	 Val. Loss: 0.070 |  Val. Acc: 98.13%
